<a href="https://colab.research.google.com/github/skhazaei/test/blob/master/preprocessing_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import warnings
warnings.filterwarnings("ignore")
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_fscore_support, roc_curve, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import  DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id = '1ExjM-usGlSbW2HUexdEFkhfwebr4243-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('diabetic_data.csv')

In [7]:
data = pd.read_csv('diabetic_data.csv')

In [9]:
data.shape

(101766, 50)

2. Dealing with Duplicate Records : We found out that, for some patient there are number of entries in the dataset these duplicate entries are not helpful in solving the task hence we removed these entries.

In [10]:
print('Total data = ', len(data))
print('Unique entries = ', len(np.unique(data['patient_nbr'])))
unique_patients = data.drop_duplicates(['patient_nbr'], keep = 'first')
unique_patients.shape

Total data =  101766
Unique entries =  71518


(71518, 50)

In [11]:
#data.dropna(inplace = True)
#print('Total data = ', len(data))
#print('Unique entries = ', len(np.unique(data['patient_nbr'])))
data.drop_duplicates(['patient_nbr'], keep = 'first', inplace = True)
#print('Length after removing Duplicates:', len(data))

In [12]:
data.shape

(71518, 50)

4. Dealing with Missing Values : Missing values in this dataset are represented by ‘?’. Only very few of the columns have missing values. These are, ‘race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3'

In [13]:
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in data.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+--------------------------+----------------+-----------------------+
|       Column Name        | Missing Values |   Missing Percentage  |
+--------------------------+----------------+-----------------------+
|       encounter_id       |       0        |          0.0          |
|       patient_nbr        |       0        |          0.0          |
|           race           |      1948      |  0.027237898151514305 |
|          gender          |       0        |          0.0          |
|           age            |       0        |          0.0          |
|          weight          |     68665      |   0.960107944853044   |
|    admission_type_id     |       0        |          0.0          |
| discharge_disposition_id |       0        |          0.0          |
|   admission_source_id    |       0        |          0.0          |
|     time_in_hospital     |       0        |          0.0          |
|        payer_code        |     31043      |   0.434058558684527   |
|    medical_special

In [14]:
columns_with_missed_values = data[['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']]

In [15]:
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------------+----------------+-----------------------+
|    Column Name    | Missing Values |   Missing Percentage  |
+-------------------+----------------+-----------------------+
|        race       |      1948      |  0.027237898151514305 |
|       weight      |     68665      |   0.960107944853044   |
|     payer_code    |     31043      |   0.434058558684527   |
| medical_specialty |     34477      |   0.4820744427976174  |
|       diag_1      |       11       | 0.0001538074330937666 |
|       diag_2      |      294       | 0.0041108532117788525 |
|       diag_3      |      1225      |  0.017128555049078554 |
+-------------------+----------------+-----------------------+


Drop columns that are missing too much data

In [16]:
data = data.drop(['weight', 'payer_code'], axis=1)

Remove null values Drop the rows where at least one element is missing.

In [17]:
data.dropna(inplace = True)
print('Total data = ', len(data))


Total data =  71518


In [18]:
data.shape

(71518, 48)

In [20]:
columns_with_missed_values = data[['race', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']]

In [21]:
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------------+----------------+-----------------------+
|    Column Name    | Missing Values |   Missing Percentage  |
+-------------------+----------------+-----------------------+
|        race       |      1948      |  0.027237898151514305 |
| medical_specialty |     34477      |   0.4820744427976174  |
|       diag_1      |       11       | 0.0001538074330937666 |
|       diag_2      |      294       | 0.0041108532117788525 |
|       diag_3      |      1225      |  0.017128555049078554 |
+-------------------+----------------+-----------------------+


In [22]:
print(data.shape)
print('% data retained out of unique patients: ' + str(round(len(data.index)/71518 * 100,2)) + '%')
print('% data retained out of all encounters: ' + str(round(len(data.index)/101766 * 100,2)) + '%')
print(data.columns)

(71518, 48)
% data retained out of unique patients: 100.0%
% data retained out of all encounters: 70.28%
Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtyp

Drop other columns without relevant information

In [23]:
data = data.drop(['encounter_id', 'patient_nbr'], axis=1)
data.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [24]:
data.shape

(71518, 46)

1. Age Feature : The values of age are given like [0–10),[10–20),[20–30) The age feature is not categorical but ordinal and hence we should convert it to numbers and treat it as numeric feature. We are converting age feature to numeric feature like below, [0–10) → 5, [10–20) → 15, [20–30) — -> 25

In [25]:
replaceDict = {'[0-10)' : 5,
'[10-20)' : 15,
'[20-30)' : 25, 
'[30-40)' : 35, 
'[40-50)' : 45, 
'[50-60)' : 55,
'[60-70)' : 65, 
'[70-80)' : 75,
'[80-90)' : 85,
'[90-100)' : 95}

data['age'] = data['age'].apply(lambda x : replaceDict[x])
print(data['age'].head())

0     5
1    15
2    25
3    35
4    45
Name: age, dtype: int64


6.2) Using domain knowledge on diag_1, diag_2, diag_3 : The diag_1 , diag_2 , diag_3 are categorical features but these features have lots unique values if we apply one hot encoding directly it will results in lots of features that will eventually invoke “curse of dimensionality” problem We will use the domain knowledge provided in the description file to create fewer categories.

Circulatory → 390–459, 785 → Diseases of the circulatory system
Respiratory → 460–519, 786 → Diseases of the respiratory system
Digestive → 520–579, 787 → Diseases of the digestive system
Diabetes → 250.xx → Diabetes mellitus
Injury → 800–999 → Injury and poisoning
Musculoskeletal → 710–739 → Diseases of the musculoskeletal system and connective tissue
Genitourinary → 580–629, 788 → Diseases of the genitourinary system
Neoplasms → 140–239 → Neoplasms
Pregnecy → 630–679 → Complications of pregnancy, childbirth, and the puerperium
Other
We added Pregnancy as one category original description file kept it in ‘Other’ category but from research we found that Pregnancy creates unique type of diabetes and hence it make sense to keep it as separate category.

In [29]:
print("number of entries for diag_1= ", len((data['diag_1'])))  
print("number of unique entries for diag_1= ", len(np.unique(data['diag_1'])))  
print("number of entries for diag_2= ", len((data['diag_2'])))  
print("number of unique entries for diag_2= ", len(np.unique(data['diag_2'])))  
print("number of entries for diag_3= ", len((data['diag_3'])))  
print("number of entries for diag_3= ", len(np.unique(data['diag_3'])))

number of entries for diag_1=  71518
number of unique entries for diag_1=  697
number of entries for diag_2=  71518
number of unique entries for diag_2=  726
number of entries for diag_3=  71518
number of entries for diag_3=  759


In [31]:
columns_with_missed_values = data[['diag_1', 'diag_2', 'diag_3']]
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------+----------------+-----------------------+
| Column Name | Missing Values |   Missing Percentage  |
+-------------+----------------+-----------------------+
|    diag_1   |       11       | 0.0001538074330937666 |
|    diag_2   |      294       | 0.0041108532117788525 |
|    diag_3   |      1225      |  0.017128555049078554 |
+-------------+----------------+-----------------------+


In [32]:
diag_1 = Counter(list(data['diag_1'])).most_common(1)[0][0]
diag_2 = Counter(list(data['diag_2'])).most_common(1)[0][0]
diag_3 = Counter(list(data['diag_3'])).most_common(1)[0][0]
data['diag_1'] = data['diag_1'].apply(lambda x : diag_1 if x == '?' else x)
data['diag_2'] = data['diag_2'].apply(lambda x : diag_2 if x == '?' else x)
data['diag_3'] = data['diag_3'].apply(lambda x : diag_3 if x == '?' else x)

In [33]:
columns_with_missed_values = data[['diag_1', 'diag_2', 'diag_3']]
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------+----------------+--------------------+
| Column Name | Missing Values | Missing Percentage |
+-------------+----------------+--------------------+
|    diag_1   |       0        |        0.0         |
|    diag_2   |       0        |        0.0         |
|    diag_3   |       0        |        0.0         |
+-------------+----------------+--------------------+


The diag_1 , diag_2 , diag_3 are categorical features but they have lots unique values if we apply one hot encoding directely it will results in lots of feature values that will eventually invoke "curse of dimensionality" problem We will use the domain knowledge provided in the description of data to create fewer values.

In [34]:
# Replace diagnoses with category names
#Circulatory: 390-459, 785
#Respiratory: 460-519, 786
#Digestive: 520-579, 787
#Diabetes: 250.xx
#Injury: 800-999
#Musculoskeletal: 710-739
#Genitourinary: 580-629, 788
#Neoplasms: 140-239, 
#Other: 
#780, 781, 784, 790-799, 240-279 except 250, 680-709, 782, 001-139, 290-319
#E-V (beginning with a letter), 280-289, 320-359, 630-679, 360-389, 740-759

def categorize_diagnosis(diag):
    if diag.isnumeric():
        diag = int(diag)
        if diag in range(390,460) or diag==785:
            return 'Circulatory'
        elif diag in range(460,520) or diag==786:
            return 'Respiratory'
        elif diag in range(520,580) or diag==787:
            return 'Digestive'
        elif diag in np.arange(250,260,0.01):
            return 'Diabetes'
        elif diag in range(800,1000):
            return 'Injury'
        elif diag in range(710,740):
            return  'Musculoskeletal'
        elif diag in range(580,630) or diag==788:
            return  'Genitourinary'
        elif diag in range(140,240):
            return  'Neoplasms'
        else:
            return 'Other'
    else:
        if "250" in diag:
            return 'Diabetes'
        else:
            return  'Other'

In [35]:
data['diag_1_desc'] = data.apply(lambda row: categorize_diagnosis(row['diag_1']), axis=1)
data['diag_2_desc'] = data.apply(lambda row: categorize_diagnosis(row['diag_2']), axis=1)
data['diag_3_desc'] = data.apply(lambda row: categorize_diagnosis(row['diag_3']), axis=1)

# Remove original diagnosis features
data.drop(columns=['diag_1', 'diag_2', 'diag_3'], inplace=True)
data.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'diag_1_desc', 'diag_2_desc', 'diag_3_desc'],
      dtype='object')

In [36]:
data.shape

(71518, 46)

In [38]:
print("number of entries for diag_1= ", len((data['diag_1_desc'])))  
print("number of unique entries for diag_1= ", len(np.unique(data['diag_1_desc'])))  
print("number of entries for diag_2= ", len((data['diag_2_desc'])))  
print("number of unique entries for diag_2= ", len(np.unique(data['diag_2_desc'])))  
print("number of entries for diag_3= ", len((data['diag_3_desc'])))  
print("number of entries for diag_3= ", len(np.unique(data['diag_3_desc'])))

number of entries for diag_1=  71518
number of unique entries for diag_1=  9
number of entries for diag_2=  71518
number of unique entries for diag_2=  9
number of entries for diag_3=  71518
number of entries for diag_3=  9


3- Drop encounters that resulted in discharge due to hospice or patient death Discharge Disposition ID corresponding to [11 or 13 or 14 or 19 or 20 or 21] indicates patient has expired so there is no chance that it will readmit again so we will remove these records.

Discharge Disposition ID has lots of distinct values using domain knowledge we will convert them into small number of categories.

ddi_cat1 = [6, 8, 9, 13] ddi_cat2 = [3, 4, 5, 14, 22, 23, 24] ddi_cat10 = [12, 15, 16, 17] ddi_cat11 = [19, 20, 21] ddi_cat18 = [25, 26]

In [39]:
print("number of entries for discharge_disposition_id= ", len((data['discharge_disposition_id'])))  
print("number of unique entries for discharge_disposition_id= ", len(np.unique(data['discharge_disposition_id'])))  

number of entries for discharge_disposition_id=  71518
number of unique entries for discharge_disposition_id=  26


In [40]:
data['discharge_disposition_id'] = data['discharge_disposition_id'].apply(lambda x : 1 if int(x) in [6, 8, 9, 13] 
                                                                           else ( 2 if int(x) in [3, 4, 5, 14, 22, 23, 24]
                                                                           else ( 10 if int(x) in [12, 15, 16, 17]
                                                                           else ( 11 if int(x) in [19, 20, 21]
                                                                           else ( 18 if int(x) in [25, 26] 
                                                                           else int(x) )))))

data = data[~data.discharge_disposition_id.isin([11,13,14,19,20,21])]

In [41]:
print("number of entries for discharge_disposition_id= ", len((data['discharge_disposition_id'])))  
print("number of unique entries for discharge_disposition_id= ", len(np.unique(data['discharge_disposition_id'])))  

number of entries for discharge_disposition_id=  70434
number of unique entries for discharge_disposition_id=  7


Admission Type ID :

Similarly Admission Type ID can be converted into small number of categories ati_cat1 = [2, 7] ati_cat5 = [6, 8]

In [42]:
print("number of entries for admission_type_id= ", len((data['admission_type_id'])))  
print("number of unique entries for admission_type_id= ", len(np.unique(data['admission_type_id'])))  

number of entries for admission_type_id=  70434
number of unique entries for admission_type_id=  8


In [43]:
data['admission_type_id'] = data['admission_type_id'].apply(lambda x : 1 if int(x) in [2, 7]
                                                            else ( 5 if int(x) in [6, 8]
                                                            else int(x) ))

In [44]:
print("number of entries for admission_type_id= ", len((data['admission_type_id'])))  
print("number of unique entries for admission_type_id= ", len(np.unique(data['admission_type_id'])))  

number of entries for admission_type_id=  70434
number of unique entries for admission_type_id=  4


Admission Source ID :

Similarly Admission Source ID can be converted into small number of categories asi_cat1 = [2, 3] ati_cat4 = [5, 6, 10, 22, 25] ati_cat9 = [15, 17, 20, 21] ati_cat11 = [13, 17]

In [46]:
print("number of entries for admission_source_id= ", len((data['admission_source_id'])))  
print("number of unique entries for admission_source_id= ", len(np.unique(data['admission_source_id'])))  

number of entries for admission_source_id=  70434
number of unique entries for admission_source_id=  17


In [48]:
data['admission_source_id'] = data['admission_source_id'].apply(lambda x : 1 if int(x) in [2, 3]
                                                            else ( 4 if int(x) in [5, 6, 10, 22, 25]
                                                            else ( 9 if int(x) in [15, 17, 20, 21]
                                                            else ( 11 if int(x) in [13, 14]
                                                            else int(x) ))))

In [49]:
print("number of entries for admission_source_id= ", len((data['admission_source_id'])))  
print("number of unique entries for admission_source_id= ", len(np.unique(data['admission_source_id'])))  

number of entries for admission_source_id=  70434
number of unique entries for admission_source_id=  6


In [50]:
id2 = '1P9utjV8bW_-2jlJjLXpF-h_HqiyOhPUz'
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('discharge.csv')

In [52]:
discharge = pd.read_csv("discharge.csv")

In [53]:
id3 = '1CWdb1VPP3PNQIoPKbK0yHxXTygCpvj1l'
downloaded = drive.CreateFile({'id':id3}) 
downloaded.GetContentFile('admission_type.csv')

In [54]:
admission_type = pd.read_csv("admission_type.csv")


In [55]:
admission_type

,admission_type_id,description
0,0,ignore
1,1,Emergency
2,2,Urgent
3,3,Elective
4,4,Newborn
5,5,Not Available
6,6,NaN
7,7,Trauma Center
8,8,Not Mapped


In [56]:
id4 = '1yA37H6wjJwmlKiJfRTWHGBlXXRJ3VpY_'
download = drive.CreateFile({'id': id4})
download.GetContentFile("admission_source.csv")

In [58]:
admission_source = pd.read_csv("admission_source.csv")

Add Three Columns of for IDs Mapping

In [59]:
data['admission_type'] = data['admission_type_id'].map(admission_type['description'])

In [60]:
data[['admission_type', 'admission_type_id']]


,admission_type,admission_type_id
0,Not Available,5
1,Emergency,1
2,Emergency,1
3,Emergency,1
4,Emergency,1
...,...,...
101754,Emergency,1
101755,Emergency,1
101756,Emergency,1
101758,Emergency,1


In [61]:
data['admission_source'] = data['admission_source_id'].map(admission_source['description'])


In [62]:
data[['admission_source', 'admission_source_id']]

,admission_source,admission_source_id
0,Physician Referral,1
1,Emergency Room,7
2,Emergency Room,7
3,Emergency Room,7
4,Emergency Room,7
...,...,...
101754,Emergency Room,7
101755,Emergency Room,7
101756,Emergency Room,7
101758,Emergency Room,7


In [63]:
data['discharge_disposition'] = data['discharge_disposition_id'].map(discharge['description'])

In [64]:
data.shape

(70434, 49)

lets analyse each and every feature one by one : lets remove them

In [65]:
data.drop(data[data.gender == 'Unknown/Invalid'].index, inplace = True)

In [66]:
data.shape

(70431, 49)

Glucose Serum test : A blood glucose test is used to find out if your blood sugar levels are in the healthy range. It is often used to help diagnose and monitor diabetes.

'>200' : 200 = indicates diabetes '>300' : 300 = Indicates diabetes 'Norm' : 100 = Normal 'None' : 0 = test was not taken

In [68]:
print("number of entries for max_glu_serum= ", len((data['max_glu_serum'])))  
print("number of unique entries for max_glu_serum= ", len(np.unique(data['max_glu_serum'])))  

number of entries for max_glu_serum=  70431
number of unique entries for max_glu_serum=  4


In [69]:
data['max_glu_serum'] = data['max_glu_serum'].apply(lambda x : 200 if x == '>200' 
                                                            else ( 300 if x == '>300'                                                          
                                                            else ( 100 if x == 'Norm'
                                                            else  0)))

In [70]:
print("number of entries for max_glu_serum= ", len((data['max_glu_serum'])))  
print("number of unique entries for max_glu_serum= ", len(np.unique(data['max_glu_serum'])))  

number of entries for max_glu_serum=  70431
number of unique entries for max_glu_serum=  1


A1C test : An A1C test is a blood test that reflects your average blood glucose levels over the past 3 months

'>7' : 7 '>8' : 8 Norm : 5 = Normal None : 0 = Test was not taken

In [71]:
print("number of entries for A1Cresult= ", len((data['A1Cresult'])))  
print("number of unique entries for A1Cresult= ", len(np.unique(data['A1Cresult'])))  

number of entries for A1Cresult=  70431
number of unique entries for A1Cresult=  4


In [72]:
data['A1Cresult'] = data['A1Cresult'].apply(lambda x : 7 if x == '>7' 
                                                         else (8 if  x == '>8'                                                        
                                                         else ( 5 if x == 'Norm'
                                                         else  0)))

In [73]:
print("number of entries for A1Cresult= ", len((data['A1Cresult'])))  
print("number of unique entries for A1Cresult= ", len(np.unique(data['A1Cresult'])))  

number of entries for A1Cresult=  70431
number of unique entries for A1Cresult=  4


In [77]:
print("number of entries for change= ", len((data['change'])))  
print("number of unique entries for change= ", len(np.unique(data['change'])))  
print(data['change'].value_counts())

number of entries for change=  70431
number of unique entries for change=  2
-1    38769
 1    31662
Name: change, dtype: int64


In [78]:
data['change'] = data['change'].apply(lambda x : 1 if x == 'Ch'
                                                 else -1)

In [79]:
print("number of entries for change= ", len((data['change'])))  
print("number of unique entries for change= ", len(np.unique(data['change'])))  
print(data['change'].value_counts())

number of entries for change=  70431
number of unique entries for change=  1
-1    70431
Name: change, dtype: int64


In [80]:
print("number of entries for diabetesMed= ", len((data['diabetesMed'])))  
print("number of unique entries for diabetesMed= ", len(np.unique(data['diabetesMed'])))  
print(data['diabetesMed'].value_counts())

number of entries for diabetesMed=  70431
number of unique entries for diabetesMed=  2
Yes    53606
No     16825
Name: diabetesMed, dtype: int64


In [81]:
data['diabetesMed'] = data['diabetesMed'].apply(lambda x : -1 if x == 'No'
                                                else 1)

In [82]:
print("number of entries for diabetesMed= ", len((data['diabetesMed'])))  
print("number of unique entries for diabetesMed= ", len(np.unique(data['diabetesMed'])))  
print(data['diabetesMed'].value_counts())

number of entries for diabetesMed=  70431
number of unique entries for diabetesMed=  2
 1    53606
-1    16825
Name: diabetesMed, dtype: int64


In [84]:
print("number of entries for race= ", len((data['race'])))  
print("number of unique entries for race= ", len(np.unique(data['race'])))  
print(data['race'].value_counts())

number of entries for race=  70431
number of unique entries for race=  5
Caucasian          54582
AfricanAmerican    12692
Hispanic            1506
Other               1160
Asian                491
Name: race, dtype: int64


In [83]:
data['race'] = data['race'].apply(lambda x : np.nan if x == '?' else x)
data["race"].fillna(data["race"].mode()[0], inplace = True)

In [85]:
print("number of entries for race= ", len((data['race'])))  
print("number of unique entries for race= ", len(np.unique(data['race'])))  
print(data['race'].value_counts())

number of entries for race=  70431
number of unique entries for race=  5
Caucasian          54582
AfricanAmerican    12692
Hispanic            1506
Other               1160
Asian                491
Name: race, dtype: int64


In [86]:
columns_with_missed_values = data[['race']]
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------+----------------+--------------------+
| Column Name | Missing Values | Missing Percentage |
+-------------+----------------+--------------------+
|     race    |       0        |        0.0         |
+-------------+----------------+--------------------+


In [87]:
data.shape

(70431, 49)

In [88]:
data.drop(columns=['admission_type_id', 'admission_source_id', 'discharge_disposition_id'], inplace=True)

In [89]:
data.shape

(70431, 46)

6.1) Using Domain knowledge on Medical Speciality Feature :

After research we found that medical_specialty is important feature however it has too many distinct values so when we apply one hot encoding it will unnecessarily create lots of features. To group them into smaller number of categories we used frequency based approach and domain knowledge like all kind of surgeons should be placed under ‘surgeon’ category. We failed to group below medical specialty so we grouped them into ‘ungrouped’ category.

Endocrinology →glands
Gastroenterology →stomach
Gynecology →women reproduction system
Hematology →Blood
Hematology/Oncology →Blood
Hospitalist →one who takes care of admitted patients
Oncology →cancer
Ophthalmology →eye
otolaryngology →ears
nose, and throat
Pulmonology →respiratory
Radiology — diagnosing and treating injuries and diseases using medical imaging (radiology) procedures (exams/tests) such as X-rays

In [90]:
print("number of entries for medical_specialty= ", len((data['medical_specialty'])))  
print("number of unique entries for medical_specialty= ", len(np.unique(data['medical_specialty'])))  
print(data['medical_specialty'].value_counts())

number of entries for medical_specialty=  70431
number of unique entries for medical_specialty=  71
?                                   33935
InternalMedicine                    10707
Family/GeneralPractice               5012
Emergency/Trauma                     4406
Cardiology                           4219
                                    ...  
SportsMedicine                          1
Dermatology                             1
Proctology                              1
Surgery-PlasticwithinHeadandNeck        1
Resident                                1
Name: medical_specialty, Length: 71, dtype: int64


In [91]:
columns_with_missed_values = data[['medical_specialty']]
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------------+----------------+--------------------+
|    Column Name    | Missing Values | Missing Percentage |
+-------------------+----------------+--------------------+
| medical_specialty |     33935      | 0.4818190853459414 |
+-------------------+----------------+--------------------+


In [93]:
high_frequency = ['InternalMedicine', 'Family/GeneralPractice', 'Cardiology', 'Surgery-General', 'Orthopedics', 'Orthopedics-Reconstructive', 
                 'Emergency/Trauma', 'Urology','ObstetricsandGynecology','Psychiatry','Pulmonology ','Nephrology','Radiologist']

low_frequency = ['Surgery-PlasticwithinHeadandNeck','Psychiatry-Addictive','Proctology','Dermatology','SportsMedicine','Speech','Perinatology',\
                'Neurophysiology','Resident','Pediatrics-Hematology-Oncology','Pediatrics-EmergencyMedicine','Dentistry','DCPTEAM','Psychiatry-Child/Adolescent',\
                'Pediatrics-Pulmonology','Surgery-Pediatric','AllergyandImmunology','Pediatrics-Neurology','Anesthesiology','Pathology','Cardiology-Pediatric',\
                'Endocrinology-Metabolism','PhysicianNotFound','Surgery-Colon&Rectal','OutreachServices',\
                'Surgery-Maxillofacial','Rheumatology','Anesthesiology-Pediatric','Obstetrics','Obsterics&Gynecology-GynecologicOnco']

pediatrics = ['Pediatrics','Pediatrics-CriticalCare','Pediatrics-EmergencyMedicine','Pediatrics-Endocrinology','Pediatrics-Hematology-Oncology',\
               'Pediatrics-Neurology','Pediatrics-Pulmonology', 'Anesthesiology-Pediatric', 'Cardiology-Pediatric', 'Surgery-Pediatric']

psychic = ['Psychiatry-Addictive', 'Psychology', 'Psychiatry',  'Psychiatry-Child/Adolescent', 'PhysicalMedicineandRehabilitation', 'Osteopath']


neurology = ['Neurology', 'Surgery-Neuro',  'Pediatrics-Neurology', 'Neurophysiology']


surgery = ['Surgeon', 'Surgery-Cardiovascular', \
          'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', \
             'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck',  'Surgery-Thoracic',\
             'Surgery-Vascular', 'SurgicalSpecialty', 'Podiatry']
             
ungrouped = ['Endocrinology','Gastroenterology','Gynecology','Hematology','Hematology/Oncology','Hospitalist','InfectiousDiseases',\
           'Oncology','Ophthalmology','Otolaryngology','Pulmonology','Radiology']

In [94]:
missing = ['?']

colMedical = []

for val in data['medical_specialty'] :
    if val in pediatrics :
        colMedical.append('pediatrics')
    elif val in psychic :
        colMedical.append('psychic')
    elif val in neurology :
        colMedical.append('neurology')
    elif val in surgery :
        colMedical.append('surgery')
    elif val in high_frequency :
        colMedical.append('high_freq')
    elif val in low_frequency :
        colMedical.append('low_freq')
    elif val in ungrouped :
        colMedical.append('ungrouped')
    elif val in missing :
        colMedical.append('missing')

In [96]:
data['medical_specialty'] = colMedical

In [97]:
print("number of entries for medical_specialty= ", len((data['medical_specialty'])))  
print("number of unique entries for medical_specialty= ", len(np.unique(data['medical_specialty'])))  
print(data['medical_specialty'].value_counts())

number of entries for medical_specialty=  70431
number of unique entries for medical_specialty=  8
missing       33935
high_freq     29264
surgery        3408
ungrouped      1786
psychic         904
neurology       573
pediatrics      460
low_freq        101
Name: medical_specialty, dtype: int64


In [98]:
columns_with_missed_values = data[['medical_specialty']]
t = PrettyTable(['Column Name', 'Missing Values', 'Missing Percentage'])
for col in columns_with_missed_values.columns :
    lenn = len(data[data[col] == '?'])
    t.add_row([col,lenn,lenn / len(data)])
print(t)

+-------------------+----------------+--------------------+
|    Column Name    | Missing Values | Missing Percentage |
+-------------------+----------------+--------------------+
| medical_specialty |       0        |        0.0         |
+-------------------+----------------+--------------------+


In [99]:
# List categorical and numeric features
categorical_feat = []
numerical_feat = []

for col in data.columns:
    if data[col].dtype == 'object':
        categorical_feat.append(col)
    elif 'id' in col:
        categorical_feat.append(col)
    elif data[col].dtype == 'int64':
        numerical_feat.append(col)

In [100]:
categorical_feat

['race',
 'gender',
 'medical_specialty',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'readmitted',
 'diag_1_desc',
 'diag_2_desc',
 'diag_3_desc',
 'admission_type',
 'admission_source',
 'discharge_disposition']

In [105]:
len(categorical_feat)

33

In [108]:
rejected_features = []
for col in categorical_feat:
    rho , pval = scipy.stats.spearmanr(data['readmitted'], data[col])
    print(col, rho, pval)
    

print('\n\n')
for col in categorical_feat:
    rho , pval = scipy.stats.spearmanr(data['readmitted'], data[col])
    if pval < 0.4 : 
        print(col, 'is significant')
    else : 
        print(col, 'is not significant')
        rejected_features.append(col)

#rejected_features.remove('max_glu_serum')
#rejected_features.remove('A1Cresult')
print(rejected_features)
## lets remove rejected_features
data.drop(rejected_features, axis = 1, inplace=True)

race -0.0056930216776534285 0.13082735764567194
gender 0.015207461749188637 5.435965568314954e-05
medical_specialty 0.0003131693531934016 0.9337640436064796
metformin 0.022281542689072738 3.3411697625448246e-09
repaglinide -0.021595125497127762 9.945112266553787e-09
nateglinide -0.003605434400493811 0.3386547964799904
chlorpropamide -0.0013294131214496322 0.7242348537760817
glimepiride -0.0007267309783378537 0.8470665993443142
acetohexamide -0.003791400641516911 0.31432923753633707
glipizide -0.020623639175723112 4.405869901071218e-08
glyburide -0.0051329602568039425 0.17313071278812506
tolbutamide 0.0018519647679088576 0.6230858239143404
pioglitazone -0.0111034557191278 0.003211152525857049
rosiglitazone -0.01968313661479235 1.7501424961250147e-07
acarbose -0.00929070562182101 0.013676302823463946
miglitol -0.005250246444981802 0.16351719721882754
troglitazone -0.00262378419144524 0.4862342646322352
tolazamide 0.0036183467842155095 0.33692772504819735
examide nan nan
citoglipton nan n

In [101]:
numerical_feat

['age',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'change',
 'diabetesMed']

In [106]:
len(numerical_feat)

13

In [107]:
rejected_features = []
for col in numerical_feat:
    rho , pval = scipy.stats.spearmanr(data['readmitted'], data[col])
    print(col, rho, pval)
    

print('\n\n')
for col in numerical_feat:
    rho , pval = scipy.stats.spearmanr(data['readmitted'], data[col])
    if pval < 0.4 : 
        print(col, 'is significant')
    else : 
        print(col, 'is not significant')
        rejected_features.append(col)

#rejected_features.remove('max_glu_serum')
#rejected_features.remove('A1Cresult')
print(rejected_features)
## lets remove rejected_features
data.drop(rejected_features, axis = 1, inplace=True)

age -0.07429678582673846 8.954111224319551e-87
time_in_hospital -0.07379883754980403 1.2272882383771764e-85
num_lab_procedures -0.05609698877436465 3.347612421210297e-50
num_procedures 0.02478765771921836 4.7282484355105157e-11
num_medications -0.06365929179245991 3.714800263687477e-64
number_outpatient -0.07997419682499037 2.7859885033461646e-100
number_emergency -0.08702137158913922 1.9221892391523947e-118
number_inpatient -0.14978698985912295 0.0
number_diagnoses -0.10244492657140511 1.3016114001998071e-163
max_glu_serum nan nan
A1Cresult 0.019212578630065952 3.411361276730968e-07
change nan nan
diabetesMed -0.061321611132351524 1.1798102813483178e-59



age is significant
time_in_hospital is significant
num_lab_procedures is significant
num_procedures is significant
num_medications is significant
number_outpatient is significant
number_emergency is significant
number_inpatient is significant
number_diagnoses is significant
max_glu_serum is not significant
A1Cresult is significant
c